In [2]:
import numpy as np
import pandas as pd
import pylab as pl
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col

In [3]:
# df5 = pd.read_csv("/Users/apple/Desktop/5003-project/moviedata/TMD/movies_metadata.csv")
df = spark.read.csv("/Users/apple/Desktop/5003-project/moviedata/TMD/movies_metadata.csv", header=True, inferSchema=True,multiLine=True,escape='"')

In [4]:
# df5.drop(df5.index[19730],inplace=True)
# df5.drop(df5.index[29502],inplace=True)
# df5.drop(df5.index[35585],inplace=True)
df

DataFrame[adult: string, belongs_to_collection: string, budget: string, genres: string, homepage: string, id: string, imdb_id: string, original_language: string, original_title: string, overview: string, popularity: string, poster_path: string, production_companies: string, production_countries: string, release_date: string, revenue: bigint, runtime: double, spoken_languages: string, status: string, tagline: string, title: string, video: boolean, vote_average: double, vote_count: int]

In [5]:
# 取出数据,转换类型
df_numeric = df.select(df['id'].astype('int'),df['budget'].astype('int'),df['popularity'].astype('double'),'revenue','runtime','vote_average','vote_count','title')
# 删除空白行,去掉几个值为零的
dfn = df_numeric.dropna()
dfn1 = dfn.where(col('budget')!=0).where((col('revenue')!=0)&(col('runtime')!=0)&(col('popularity')!=0)&(col('vote_average')!=0)&(col('vote_count')>100)).select('id','budget','popularity','revenue','runtime','vote_average','vote_count','title')
dfn1.show()

+-----+--------+----------+---------+-------+------------+----------+--------------------+
|   id|  budget|popularity|  revenue|runtime|vote_average|vote_count|               title|
+-----+--------+----------+---------+-------+------------+----------+--------------------+
|  862|30000000| 21.946943|373554033|   81.0|         7.7|      5415|           Toy Story|
| 8844|65000000| 17.015539|262797249|  104.0|         6.9|      2413|             Jumanji|
|  949|60000000| 17.924927|187436818|  170.0|         7.7|      1886|                Heat|
| 9091|35000000|   5.23158| 64350171|  106.0|         5.5|       174|        Sudden Death|
|  710|58000000| 14.686036|352194034|  130.0|         6.6|      1194|           GoldenEye|
| 9087|62000000|  6.318445|107879496|  106.0|         6.5|       199|The American Pres...|
| 1408|98000000|  7.284477| 10017322|  119.0|         5.7|       137|    Cutthroat Island|
|  524|52000000| 10.137389|116112375|  178.0|         7.8|      1343|              Casino|

In [6]:
dfn1.describe('id','budget','popularity','revenue').show()
dfn1.describe('id','runtime','vote_average','vote_count').show()

+-------+-----------------+-------------------+------------------+--------------------+
|summary|               id|             budget|        popularity|             revenue|
+-------+-----------------+-------------------+------------------+--------------------+
|  count|             3793|               3793|              3793|                3793|
|   mean|58722.31373582916|3.992022711890324E7|12.145752633799113|1.2303170756208806E8|
| stddev|96257.46345259793|4.428563593487887E7|15.723544794986447|1.8778476537694427E8|
|    min|                5|                  1|          0.338943|                   3|
|    max|           419430|          380000000|        547.488298|          2787965087|
+-------+-----------------+-------------------+------------------+--------------------+

+-------+-----------------+------------------+------------------+------------------+
|summary|               id|           runtime|      vote_average|        vote_count|
+-------+-----------------+----------

In [7]:
dfn2 = dfn1.drop('title')
dfn2.show()

+-----+--------+----------+---------+-------+------------+----------+
|   id|  budget|popularity|  revenue|runtime|vote_average|vote_count|
+-----+--------+----------+---------+-------+------------+----------+
|  862|30000000| 21.946943|373554033|   81.0|         7.7|      5415|
| 8844|65000000| 17.015539|262797249|  104.0|         6.9|      2413|
|  949|60000000| 17.924927|187436818|  170.0|         7.7|      1886|
| 9091|35000000|   5.23158| 64350171|  106.0|         5.5|       174|
|  710|58000000| 14.686036|352194034|  130.0|         6.6|      1194|
| 9087|62000000|  6.318445|107879496|  106.0|         6.5|       199|
| 1408|98000000|  7.284477| 10017322|  119.0|         5.7|       137|
|  524|52000000| 10.137389|116112375|  178.0|         7.8|      1343|
| 4584|16500000| 10.673167|135000000|  136.0|         7.2|       364|
|    5| 4000000|  9.026586|  4300000|   98.0|         6.5|       539|
| 9273|30000000|  8.205448|212385533|   90.0|         6.1|      1128|
|11517|60000000|  7.

In [8]:
for i in dfn2.columns[1:]:
    max1=float(dfn2.describe(i).filter(col('summary')=='max').collect()[0][1])
    min1=float(dfn2.describe(i).filter(col('summary')=='min').collect()[0][1])
    print(max1,min1)
    dfn2=dfn2.withColumn(i,((dfn2[i]-min1)/(max1-min1)))
dfn2.show()
dfn2 = dfn2.drop('id')
# 转换为RDD
dfn3 = dfn2.rdd

380000000.0 1.0
547.488298 0.338943
2787965087.0 3.0
248.0 26.0
9.1 2.8
14075.0 101.0
+-----+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+
|   id|              budget|          popularity|             revenue|            runtime|      vote_average|          vote_count|
+-----+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+
|  862| 0.07894736599722992| 0.03949195919275095| 0.13398805894084145|0.24774774774774774|0.7777777777777779|  0.3802776585086589|
| 8844| 0.17105262939750693|0.030479056308126324| 0.09426131177473526|0.35135135135135137|0.6507936507936509|  0.1654501216545012|
|  949| 0.15789473462603879| 0.03214110341042072| 0.06723068953613912| 0.6486486486486487|0.7777777777777779| 0.12773722627737227|
| 9091| 0.09210526076869806| 0.00894205020126543|0.023081411015260764|0.36036036036036034|0.4285714285714286| 0.0052239874051810

In [9]:
dfn4 = dfn3.map(lambda x:np.array([x[0],x[1],x[2],x[3],x[4]]))
type(dfn4)

<class 'pyspark.rdd.PipelinedRDD'>

In [10]:
dfn4.take(10)

[array([0.07894737, 0.03949196, 0.13398806, 0.24774775, 0.77777778]), array([0.17105263, 0.03047906, 0.09426131, 0.35135135, 0.65079365]), array([0.15789473, 0.0321411 , 0.06723069, 0.64864865, 0.77777778]), array([0.09210526, 0.00894205, 0.02308141, 0.36036036, 0.42857143]), array([0.15263158, 0.02622153, 0.12632656, 0.46846847, 0.6031746 ]), array([0.16315789, 0.01092846, 0.03869471, 0.36036036, 0.58730159]), array([0.25789473, 0.01269404, 0.00359306, 0.41891892, 0.46031746]), array([0.1368421 , 0.01790817, 0.04164771, 0.68468468, 0.79365079]), array([0.04342105, 0.01888739, 0.04842241, 0.4954955 , 0.6984127 ]), array([0.01052631, 0.01587801, 0.00154234, 0.32432432, 0.58730159])]

In [11]:
dfn4.count()

3793

In [12]:
from numpy import array
from math import sqrt
from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans, KMeansModel

In [13]:
# Build the model (cluster the data)
clusters = KMeans.train(dfn4, 5, maxIterations=10, initializationMode="random")
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))
WSSSE = dfn4.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))
print("Within Set Avg of Squared Error = " + str(WSSSE/3793))

Within Set Sum of Squared Error = 421.41291022270923
Within Set Avg of Squared Error = 0.11110279731682289


In [14]:
def getlabel(point):
    label=clusters.predict(point)
    return label

la=dfn4.map(lambda point:getlabel(point) )
lalist=list(la.countByValue().items())
X=[]
Y=[]
for i in lalist:
    X.append(i[0])
    Y.append(i[1])
X,Y

([4, 0, 1, 2, 3], [943, 1360, 471, 676, 343])

In [15]:
# Build the model (cluster the data)
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))
klist=[]
score=[]
for i in range(1,9):
    clusters = KMeans.train(dfn4, i, maxIterations=10, initializationMode="random")
    klist.append(i)
    
    WSSSE = dfn4.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    score.append(WSSSE)
    print("Within Set Sum of Squared Error = " + str(WSSSE))
    # print("Within Set Avg of Squared Error = " + str(WSSSE/3793))

Within Set Sum of Squared Error = 689.1982151039994
Within Set Sum of Squared Error = 593.96266802139
Within Set Sum of Squared Error = 489.658201743694
Within Set Sum of Squared Error = 451.56164526569677
Within Set Sum of Squared Error = 421.4789486661987
Within Set Sum of Squared Error = 395.31618925378973
Within Set Sum of Squared Error = 381.98484943192506
Within Set Sum of Squared Error = 365.7616532188139


In [16]:
pl.plot(klist,score)
pl.xlabel('Number of Clusters')
pl.ylabel('Score')
pl.title('Elbow Curve')

Text(0.5,1,'Elbow Curve')

In [17]:
pl.show()

In [18]:
plt.figure(figsize=(12,7))
axis = sns.barplot(x=X,y=Y)
x=axis.set_xlabel("Cluster Number")
x=axis.set_ylabel("Number of movies")

In [ ]:
pl.show()